In [95]:
import io
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt
import os
import numpy as np

In [96]:
conn = sqlite3.connect('sql/conn_trabalho.db', timeout=10)
c = conn.cursor()

In [97]:
dataB = pd.read_csv("dataset/LGBT_Survey_DailyLife.csv")
dataB.head()

,CountryCode,subset,question_code,question_label,answer,percentage,notes
0,Austria,Lesbian,b1_a,"In your opinion, how widespread is offensive l...",Very widespread,8,NaN
1,Austria,Lesbian,b1_a,"In your opinion, how widespread is offensive l...",Fairly widespread,34,NaN
2,Austria,Lesbian,b1_a,"In your opinion, how widespread is offensive l...",Fairly rare,45,NaN
3,Austria,Lesbian,b1_a,"In your opinion, how widespread is offensive l...",Very rare,9,NaN
4,Austria,Lesbian,b1_a,"In your opinion, how widespread is offensive l...",Don`t know,4,[0]


In [98]:
dataB["CountryCode"]

0        Austria
1        Austria
2        Austria
3        Austria
4        Austria
          ...   
34015    Average
34016    Average
34017    Average
34018    Average
34019    Average
Name: CountryCode, Length: 34020, dtype: object

# Criando as tabelas

In [99]:
c.execute('''CREATE TABLE COUNTRY
             (COUNTRYCODE         INT NOT NULL,
              COUNTRY             TEXT NOT NULL,
              
              PRIMARY KEY(COUNTRYCODE)
              
             )''')

In [100]:
c.execute('''CREATE TABLE QUESTIONS
             (QC                  VARCHAR(10) NOT NULL,
              QUESTION            TEXT NOT NULL,
              CATEGORY            TEXT NOT NULL,
              
              PRIMARY KEY(QC)
             )''')

In [101]:
c.execute('''CREATE TABLE ANSWERS
             (COUNTRYCODE         INT NOT NULL,
              SUBSET              TEXT NOT NULL,
              QC                  VARCHAR(10) NOT NULL,
              ANSWER              TEXT NOT NULL,
              PERCENTAGE          INT NOT NULL,
              NOTES               TEXT,
              
              PRIMARY KEY(COUNTRYCODE,SUBSET,QC,ANSWER),
              
              FOREIGN KEY(COUNTRYCODE) REFERENCES COUNTRY(COUNTRYCODE),
              FOREIGN KEY(QC) REFERENCES QUESTIONS(QC)
              
             )''')

In [102]:
c.execute('''CREATE TABLE SIZE
             (COUNTRYCODE         INT NOT NULL,
              TIPO                TEXT NOT NULL,
              VALOR               INT NOT NULL,
              
              PRIMARY KEY(COUNTRYCODE,TIPO),
              
              FOREIGN KEY(COUNTRYCODE) REFERENCES COUNTRY(COUNTRYCODE)
              
             )''')

In [103]:
c.execute('''CREATE TABLE IDH
             (COUNTRYCODE         INT NOT NULL,
              HDI                 REAL NOT NULL,
              LEB                 REAL NOT NULL,
              EYS                 REAL NOT NULL,
              MYS                 REAL NOT NULL,
              GNI                 REAL NOT NULL,
              GNI_HDI             REAL NOT NULL,
              
              PRIMARY KEY(COUNTRYCODE),
              
              FOREIGN KEY(COUNTRYCODE) REFERENCES COUNTRY(COUNTRYCODE)
              
             )''')

In [104]:
c.execute('''CREATE TABLE PESOS
             (QC                  VARCHAR(10) NOT NULL,
              ANSWER              TEXT NOT NULL,
              VALUE               INT NOT NULL,
              
              PRIMARY KEY(QC,ANSWER),
              FOREIGN KEY(QC) REFERENCES QUESTIONS(QC),
              FOREIGN KEY(ANSWER) REFERENCES QUESTIONS(ANSWER)
             )''')

In [105]:
c.execute('''CREATE TABLE CULTURE
             (COUNTRYCODE         INT NOT NULL,
              pdi                 INT,
              idv                 INT,
              mas                 INT,
              uai                 INT,
              ltowvs              INT,
              ivr                 INT,
              
              PRIMARY KEY(COUNTRYCODE),
              FOREIGN KEY(COUNTRYCODE) REFERENCES COUNTRY(COUNTRYCODE)
             )''')

In [106]:
c.execute('''CREATE TABLE Religion
             (COUNTRYCODE         INT NOT NULL,
              Catholic                 REAL,
              Orthodox                 REAL,
              Protestant                 REAL,
              OtherChristian                 REAL,
              TotalChristians              REAL,
              NonBeliever                 REAL,
              Atheist                 REAL,
              Muslim                 REAL,
              
              PRIMARY KEY(COUNTRYCODE),
              FOREIGN KEY(COUNTRYCODE) REFERENCES COUNTRY(COUNTRYCODE)
             )''')

# Variaveis auxiliares

In [107]:
conv = open("conv.txt","r")

table1 = {}

p = conv.read().split("\n")

for x in p:
    s = x.split("|")[0][:-1].upper()
    i = x.split("|")[1][1:].upper()
    
    table1[i] = s

In [109]:
conv2 = open("conv2.txt","r")

tableC2 = {}

p = conv2.read().split("\n")

for x in p:
    s = x.split("|")[0][:-1].upper()
    i = x.split("|")[1][1:].upper()
    
    tableC2[s] = i

In [110]:
tableS = {"Lesbian": "LESBIAN",
          "Gay": "GAY",
          "Bisexual women" : "BIW",
          "Bisexual men" : "BIM",
          "Transgender" : "TRANS"}

tableS2 = {"Lesbian women": "LESBIAN",
          "Gay men": "GAY",
          "Bisexual women" : "BIW",
          "Bisexual men" : "BIM",
          "Transgender" : "TRANS"}


# Populando

In [111]:
#Country

table2 = {}

data = pd.read_csv("dataset/LGBT_Survey_SubsetSize.csv")

data = data.drop(0)

data = data.sort_values("CountryID")

paises = data["CountryID"].unique()

for x in range(len(paises)):
    
    table2[paises[x]] = x
    
    querry = """INSERT INTO COUNTRY
          VALUES ({},'{}') """.format(x,paises[x])
    
    c.execute(querry)

In [112]:
#questions

code = []
for x in os.listdir("dataset"):    
    print("dataset\{}".format(x))
    data = pd.read_csv("dataset\{}".format(x))
    
    if list(data.columns).count("question_code") == 0:
        continue
    
    cat = x.split("_")[2]
    cat = cat.split(".")[0]
    
    q = data[["question_code","question_label"]].values
    
    for i in q:
        if code.count(i[0]):
            continue
        
        code.append(i[0])
        
        querry = """INSERT INTO QUESTIONS
          VALUES ('{}','{}','{}') """.format(i[0],i[1],cat)
        
        try:
            c.execute(querry)
        except:
            1+1

dataset\.~lock.religion.csv#
dataset\6-dimensions-for-website-2015-12-08-0-100.csv
dataset\human-development-index-hdi-2014.csv
dataset\LGBT_Survey_DailyLife.csv
dataset\LGBT_Survey_Discrimination.csv
dataset\LGBT_Survey_RightsAwareness.csv
dataset\LGBT_Survey_SubsetSize.csv
dataset\LGBT_Survey_TransgenderSpecificQuestions.csv
dataset\LGBT_Survey_ViolenceAndHarassment.csv
dataset\pesos.csv
dataset\religion.csv


In [113]:
#Answers

for x in os.listdir("dataset"):
    data = pd.read_csv("dataset\{}".format(x))
    
    if list(data.columns).count("question_code") == 0:
        continue
        
    data = data.fillna("NULL")
    
    for i in data.index:
        sample = data.loc[i]
        
        try:
            idC = table1[sample["CountryCode"].upper()]
            idC = table2[idC]
            
        except:
            continue
        
        subset = tableS[sample["subset"]]
        qc = sample["question_code"]
        answer = sample["answer"]
        percentage = sample["percentage"]
        notes = sample["notes"]
        
        querry = """INSERT INTO ANSWERS
          VALUES ({},'{}','{}','{}',{},'{}') """.format(idC,subset,qc,answer,percentage,notes)
        
        try:
            c.execute(querry)            
        except:
            continue

In [114]:
#SIZE

data = pd.read_csv("dataset/LGBT_Survey_SubsetSize.csv")

for x in data.index:
    sample = data.loc[x]
    
    idC = sample["CountryID"]
    
    try:
        idC = table2[idC]
    except:
        continue
        
    for hi in tableS2.keys():
        value = sample[hi]
        
        querry = """INSERT INTO SIZE
          VALUES ({},'{}',{}) """.format(idC,tableS2[hi],value)
        
        c.execute(querry)  

In [115]:
#IDH

data = pd.read_csv("dataset/human-development-index-hdi-2014.csv")

for x in range(data.shape[0]):
    sample = data.loc[x]
    
    country = sample["Location"]
    
    if list(table1.keys()).count(country.upper()) == 0:
        continue
    
    idC = table1[country.upper()]
    idC = table2[idC]
    
    sample = sample.values
    sample = sample[2:]
    
    querry = """INSERT INTO IDH
          VALUES ({}""".format(idC)
    
    for y in sample:
        if type(y) == str:
            y = y.split(",")
            y = ".".join(y)
            y =  float(y)
            
        querry += ",{}".format(y)
    
    querry += ")"
    
    c.execute(querry)

In [116]:
#PESOS

data = pd.read_csv("dataset/pesos.csv",sep=",")
for x in range(data.shape[0]):
    sample = data.loc[x]
    
    qc = sample["QC"]
    ans = sample["ANSWER"]
    valu = sample["VALUE"]
    
    querry = """INSERT INTO PESOS
          VALUES ('{}','{}',{})   """.format(qc,ans,valu)
    
    c.execute(querry)

In [117]:
#Culture

data = pd.read_csv("dataset/6-dimensions-for-website-2015-12-08-0-100.csv", sep = ";")
data = data.replace("#NULL!", "NULL")

for x in range(data.shape[0]):
    sample = data.loc[x]
    
    if list(tableC2.keys()).count(sample["ctr"]) == 0:
        continue
        
    cc = tableC2[sample["ctr"]]
    
    
    valores = sample.values[2:]
    
    querry = """INSERT INTO CULTURE
          VALUES ({}""".format(cc)
    
    for i in valores:
        querry += ", {}".format(i)
        
    querry += ")"
    
    
    c.execute(querry)
    


In [123]:
#Religion

data = pd.read_csv("dataset/religion.csv")

print(data)

for x in range(data.shape[0]):
    sample = data.loc[x]
    
    country = sample["Location"]
    
    if list(table1.keys()).count(country.upper()) == 0:
        continue
    
    idC = table1[country.upper()]
    idC = table2[idC]
    
    sample = sample.values
    sample = sample[1:]
    
    querry = """INSERT INTO Religion
          VALUES ({}""".format(idC)
    
    for y in sample:
        if type(y) == str:
            y = y.split(",")
            y = ".".join(y)
            y =  float(y)
            
        querry += ",{}".format(y)
    
    querry += ")"
    
    c.execute(querry)

            Location  Catholic  Orthodox  Protestant  Other Christian  \
0        Netherlands      21.9       1.9        17.8              6.2   
1     Czech Republic      27.1       0.2         1.0              3.2   
2             Sweden       1.6       0.9        36.5              8.6   
3      Great Britain      12.7       9.6        14.7             19.2   
4            Hungary      60.3       1.1         5.1              8.1   
5             France      47.8       0.6         1.8              4.1   
6             Latvia      26.2      24.0        16.6              9.9   
7            Belgium      52.9       1.6         2.1              4.1   
8              Spain      64.2       1.4         0.8              2.2   
9            Estonia       2.8      23.2         9.0             23.6   
10           Austria      66.5       2.2         7.2              1.9   
11              EU28      45.3       9.6        11.1              5.6   
12           Finland       0.1       2.0        69.

# Salvando

In [124]:
#backup
#Referencia: http://pythonclub.com.br/gerenciando-banco-dados-sqlite3-python-parte1.html, https://stackoverflow.com/questions/49394737/exporting-data-from-google-colab-to-local-machine

f = io.open("sql/trabalho.sql",'w')

for texto in conn.iterdump():
    f.write("{}\n".format(texto))

f.close()

In [125]:
conn.close()